In [96]:
#https://www.youtube.com/watch?v=xIqX1dqcNbY
import pandas as pd
import numpy as np

In [97]:
df=pd.read_csv('creditcard.csv')
numOfRows= len(df.index)
halfRows= (numOfRows/2)-0.5
#Use the first half as training rows and hte second half as test rows
train_df=df.head(142403)
test_df= df.tail(142403)
test_df=test_df.reset_index()

In [98]:
#Grab all the rows and all the columns up until but not included the last one. There are 2 statements for X_train and y_train respectively
X_train, y_train= train_df.to_numpy()[:, :], train_df.to_numpy()[:, -1]
X_test, y_test= test_df.to_numpy()[:, :-1], test_df.to_numpy()[:, -1]

#shows number of rows and columns for each
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((142403, 31), (142403,), (142403, 31), (142403,))

In [99]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
#standardscaler converts to min-max curve, and minmaxscaler converts to [-1, 1]
from copy import deepcopy

#use standard scaler on all rows and first 2 columns
std_scaler= StandardScaler().fit(X_train[:, :2])
min_max_scaler= MinMaxScaler().fit(X_train[:, 2:])

def preprocessor(X):
    A= np.copy(X)
    A[:, :2] = std_scaler.transform(X[:, :2])
    A[:, 2:] = min_max_scaler.transform(X[:, 2:])
    return A


In [100]:
preprocessor(X_test)

array([[ 4.26455001e+00,  4.66872664e+04,  7.88880279e-01, ...,
         2.62501153e-01, -5.84680686e-07,  1.90000000e+01],
       [ 4.26459740e+00,  4.66878176e+04,  7.40547551e-01, ...,
         2.88686214e-01, -6.50896808e-06,  1.23100000e+01],
       [ 4.26464478e+00,  4.66883689e+04,  8.04740301e-01, ...,
         2.57214365e-01,  1.56495492e-06,  6.66000000e+01],
       ...,
       [ 1.10119350e+01,  9.52497523e+04,  8.14634085e-01, ...,
         2.57148460e-01, -1.35124707e-06,  6.78800000e+01],
       [ 1.10119824e+01,  9.52497523e+04,  7.91057923e-01, ...,
         2.59439261e-01,  5.31796922e-06,  1.00000000e+01],
       [ 1.10120298e+01,  9.52519573e+04,  7.87860168e-01, ...,
         2.56997663e-01,  6.94370488e-07,  2.17000000e+02]])

In [101]:
preprocess_transformer = FunctionTransformer(preprocessor)
preprocess_transformer

FunctionTransformer(func=<function preprocessor at 0x11c5a1a60>)

In [102]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

#first pipeline
p1=Pipeline([('Scaler', preprocess_transformer), 
            ('Linear Regression', LinearRegression())])
p1

Pipeline(steps=[('Scaler',
                 FunctionTransformer(func=<function preprocessor at 0x11c5a1a60>)),
                ('Linear Regression', LinearRegression())])

In [103]:
from sklearn.metrics import mean_absolute_error

def fit_and_print(p, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    p.fit(X_train, y_train)
    train_preds= p.predict(X_train)
    test_preds=p.predict(X_test)
    print('Training Error: ' + str(mean_absolute_error(train_preds, y_train)))
    print('Test error: '+ str(mean_absolute_error(test_preds, y_test)))
    

In [104]:
fit_and_print(p1)

Training Error: 5.106677654278382e-17
Test error: 86.01152510831706


In [105]:
#the Linear Regression model is not great so try a better model: KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor as KNR

p2= Pipeline([('Scaler', preprocess_transformer), 
            ('KNN Regression', KNR(n_neighbors=7))])



In [ ]:
fit_and_print(p2)

In [ ]:
#Try Random Forest
from sklearn.ensemble import RandomForestRegressor as RFR

p3= Pipeline([('scaler', preprocess_transformer),
             ('Random Forest', RFR(n_estimators=10, max_depth=7))])


In [ ]:
fit_and_print(p3)